# EXTRACTOR DE GASTOS DE ESTADO DE CUENTA BBVA

Este es un script que permite extraer los renglones de gastos, pagos, devoluciones y meses sin intereses de un estado de cuenta de BBVA Mexico.

También intenta etiquetar algunos conceptos y crear una gráfica para visualizar en qué rubros se ha estado gastando más.

Recuerda colocar los estados de cuenta en formato PDF en el mismo directorio en el que este script se encuentra.

## 1. Primero instalamos los paquetes necesarios

In [ ]:
!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
!pip install pandas matplotlib!pip install pdftotext

## 2. Establecemos las opciones de visualización

In [ ]:
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.width = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
pd.options.display.max_info_columns = 100
pd.options.display.precision = 5
pd.options.display.float_format = '{:.1f}'.format

In [ ]:
pd.set_option('display.expand_frame_repr', False)

## 3. Definimos las reglas de extracción como expresiones regulares. Estas pueden requerir actualización, modificación o borrado dependiendo si BBVA cambia su formato de estado de cuenta.

In [ ]:
import re
import os
import pdftotext

gastos = []
devoluciones = []
mensualidades = []
meses_sin_intereses = []
pago_tarjeta = []

not_matching = []
trans_pat_1 = (
        r'(?P<date1>\d+/\d+/\d+)\s*'
        r'(?P<date2>\d+/\d+/\d+)\s*'
        r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
        r'(?P<RFC>[A-Z]{3,4}[ ]?[0-9]{6}[A-Z0-9]{3})\s*'
        r'(?P<CARD>\*{6}[0-9]{4})\s*'
        r'\$\s*'
        r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4})'
)

trans_pat_2 = (
    r'(?P<date1>\d+-\d+-\d+)\s*'
    r'(?P<date2>///[0-9]{2})\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'(?P<RFC>[A-Z]{3,4}[ ]?[0-9]{6}[A-Z0-9]{3})\s*'
    r'(?P<CARD>\*{6}[0-9]{4})\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4})'
)

trans_pat_3 = (
    r'(?P<date1>\d+/\d+/\d+)\s*'
    r'(?P<date2>\d+/\d+/\d+)\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'(?P<RFC>[A-Z]{3,4}[ ]?[0-9]{6}[A-Z0-9]{3})\s*'
    r'(?P<CARD>\*{6}[0-9]{4})\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4})'
)

trans_pat_4 = (
    r'(?P<date1>\d+/\d+/\d+)\s*'
    r'(?P<date2>\d+/\d+/\d+)\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'(?P<RFC>[A-Z]{3,4}[ ]?[0-9]{6}[A-Z0-9]{3})\s*'
    r'(?P<CARD>\*{6}[0-9]{4})\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4}-)'
)

trans_pat_5 = (
    r'(?P<date1>\d+-\d+-\d+)\s*'
    r'(?P<date2>///[0-9]{2})\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'(?P<CARD>\*{6}[0-9]{4})\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4})'
)

trans_pat_6 = (
    r'(?P<date1>\d+-\d+-\d+)\s*'
    r'(?P<date2>///[0-9]{2})\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4})'
)

trans_pat_7 = (
    r'(?P<date1>\d+\/\d+\/\d+)\s*'
    r'(?P<date2>\d+\/\d+\/\d+)\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4})'
)

trans_pat_1_2 = (
        r'(?P<date1>\d+/\d+/\d+)\s*'
        r'(?P<date2>\d+/\d+/\d+)\s*'
        r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
        r'(?P<RFC>[A-Z]{3,4}[ ]?[0-9]{6}[A-Z0-9]{3})\s*'
        r'(?P<CARD>\*{6}[0-9]{4})\s*'
        r'\$\s*'
        r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4}-)'
)

trans_pat_2_2 = (
    r'(?P<date1>\d+-\d+-\d+)\s*'
    r'(?P<date2>///[0-9]{2})\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'(?P<RFC>[A-Z]{3,4}[ ]?[0-9]{6}[A-Z0-9]{3})\s*'
    r'(?P<CARD>\*{6}[0-9]{4})\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4}-)'
)

trans_pat_3_2 = (
    r'(?P<date1>\d+/\d+/\d+)\s*'
    r'(?P<date2>\d+/\d+/\d+)\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'(?P<RFC>[A-Z]{3,4}[ ]?[0-9]{6}[A-Z0-9]{3})\s*'
    r'(?P<CARD>\*{6}[0-9]{4})\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4}-)'
)

trans_pat_5_2 = (
    r'(?P<date1>\d+-\d+-\d+)\s*'
    r'(?P<date2>///[0-9]{2})\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'(?P<CARD>\*{6}[0-9]{4})\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4}-)'
)

trans_pat_6_2 = (
    r'(?P<date1>\d+-\d+-\d+)\s*'
    r'(?P<date2>///[0-9]{2})\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4}-)'
)

trans_pat_6_3 = (
    r'(?P<date1>\d+-\d+-\d+)\s*'
    r'(?P<date2>///[0-9]{2})\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4})'
)

trans_pat_7_2 = (
    r'(?P<date1>\d+/\d+/\d+)\s*'
    r'(?P<date2>\d+/\d+/\d+)\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'\$\s*'
    r'(?P<CANTIDAD>[0-9,]{1,7}\.[0-9]{1,4}-)'
)

trans_pat_8 = (
    r'(?P<date1>\d+/\d+/\d+)\s*'
    r'(?P<description>[\.\/\w0-9\* ]+?)\s*'
    r'\$\s*(?P<TOTAL>[0-9,]{1,7}\.[0-9]{1,4})\s*'
    r'\$\s*(?P<PARCIAL>[0-9,]{1,7}\.[0-9]{1,4})\s*'
    r'(?P<MENSUALIDAD>[0-9]{2} de [0-9]{2})\s*'
    r'\$\s*'
    r'(?P<RESTANTE>[0-9,]{1,7}\.[0-9]{1,4})'
)



## 4. Comenzamos la lectura de PDFs. Si se quiere extraer sólo de un PDF, se modifica el nombre a algo más particular como 'estadoCuentaSeptiembre2024'. Si se quiere extraer lo de varios PDFs se modifica como algo más general que tengan en común los nombres de todos los PDFs como 'estadoCuenta'.

In [ ]:
nombre_archivos = 'bbvasept.pdf' #cambia el contenido de este valor de acuerdo al nombre de tus archivos
count = 0
for i in os.listdir():
  if nombre_archivos in i:
    with open(f"/content/{i}", "rb") as file:
      pdf = pdftotext.PDF(file, physical=True)
      for i, page in enumerate(pdf):
        first_page = pdf[i]
        lines = first_page.split("\n")
        for line in lines:
          count += 1
          line = line.strip()
          line = line.replace(',', '')
          match_1 = re.search(pattern=trans_pat_1, string=line)
          match_2 = re.search(pattern=trans_pat_2, string=line)
          match_3 = re.search(pattern=trans_pat_3, string=line)
          match_4 = re.search(pattern=trans_pat_4, string=line)
          match_5 = re.search(pattern=trans_pat_5, string=line)
          match_6 = re.search(pattern=trans_pat_6, string=line)
          match_7 = re.search(pattern=trans_pat_7, string=line)
          match_1_2 = re.search(pattern=trans_pat_1_2, string=line)
          match_2_2 = re.search(pattern=trans_pat_2_2, string=line)
          match_3_2 = re.search(pattern=trans_pat_3_2, string=line)
          match_5_2 = re.search(pattern=trans_pat_5_2, string=line)
          match_6_2 = re.search(pattern=trans_pat_6_2, string=line)
          match_6_3 = re.search(pattern=trans_pat_6_3, string=line)
          match_7_2 = re.search(pattern=trans_pat_7_2, string=line)
          match_8 = re.search(pattern=trans_pat_8, string=line)
          if 'S/I' in line or 'MSI' in line:
            if match_7:
              mensualidades.append(match_7.groupdict())
            elif match_6_3:
              mensualidades.append(match_6_3.groupdict())
            else:
              not_matching.append(line)
          elif 'BMOVIL.PAGO TDC' in line:
            if match_1:
              pago_tarjeta.append(match_1.groupdict())
            elif match_2:
              pago_tarjeta.append(match_2.groupdict())
            elif match_1:
              pago_tarjeta.append(match_1.groupdict())
            elif match_3:
              pago_tarjeta.append(match_3.groupdict())
            elif match_5:
              pago_tarjeta.append(match_5.groupdict())
            elif match_6:
              pago_tarjeta.append(match_6.groupdict())
            elif match_7:
              pago_tarjeta.append(match_7.groupdict())
            else:
              not_matching.append(line)
          elif match_4:
            devoluciones.append(match_4.groupdict())
          elif match_1_2:
            devoluciones.append(match_1_2.groupdict())
          elif match_2_2:
            devoluciones.append(match_2_2.groupdict())
          elif match_3_2:
            devoluciones.append(match_3_2.groupdict())
          elif match_5_2:
            devoluciones.append(match_5_2.groupdict())
          elif match_6_2:
            devoluciones.append(match_6_2.groupdict())
          elif match_7_2:
            devoluciones.append(match_7_2.groupdict())
          elif match_2:
            gastos.append(match_2.groupdict())
          elif match_1:
            gastos.append(match_1.groupdict())
          elif match_3:
            gastos.append(match_3.groupdict())
          elif match_5:
            gastos.append(match_5.groupdict())
          elif match_6:
            gastos.append(match_6.groupdict())
          elif match_7:
            gastos.append(match_7.groupdict())
          elif match_8:
            meses_sin_intereses.append(match_8.groupdict())
          else:
            not_matching.append(line)

Este es el número total de renglones revisados

In [ ]:
count

Total de gastos encontrados

In [ ]:
len(gastos)

Total de devoluciones

In [ ]:
len(devoluciones)

Total cargos de meses sin intereses detectados a pagar este mes

In [ ]:
len(mensualidades)

Total de renglones que fueron ignorados por las reglas

In [ ]:
len(not_matching)

Total de pagos a capital hechos

In [ ]:
len(pago_tarjeta)

Total de casos que se encuentran a meses sin intereses

In [ ]:
len(meses_sin_intereses)

Total de renglones categorizados, si no coincide con el count de arriba, significa que el script fallo en leer algunos renglones

In [ ]:
len(gastos)+len(devoluciones)+len(mensualidades)+len(not_matching)+len(pago_tarjeta)+len(meses_sin_intereses)

# 5. Comenzamos la carga de cada rubro:
* Gastos: Lo que pagaste con tu tarjeta
* Devoluciones: Cosas que pagaste pero que fueron devueltas a tu tarjeta
* Mensualidades: Partes x de n mensualidades a pagar este mes
* Meses sin intereses: Lista de meses sin intereses
* Pagos a tarjeta: Pagos que hiciste a tu tarjeta desde la app movil

In [ ]:
import pandas as pd
gastos = pd.DataFrame(gastos)
devoluciones = pd.DataFrame(devoluciones)
mensualidades = pd.DataFrame(mensualidades)
meses_sin_intereses = pd.DataFrame(meses_sin_intereses)
pago_tarjeta = pd.DataFrame(pago_tarjeta)

In [ ]:
gastos['CANTIDAD'] = gastos['CANTIDAD'].astype(float)
gastos['date1'] = pd.to_datetime(gastos['date1'], format='mixed')
gastos['CARD'] = gastos['CARD'].fillna('NO_CARD')
gastos['RFC'] = gastos['RFC'].fillna('NO_RFC')
if len(devoluciones) > 0:
  devoluciones['CANTIDAD'] = devoluciones['CANTIDAD'].str.replace('-', '')
  devoluciones['CANTIDAD'] = devoluciones['CANTIDAD'].astype(float)
  devoluciones['date1'] = pd.to_datetime(devoluciones['date1'], format='mixed')
  #devoluciones['CARD'] = devoluciones['CARD'].fillna('NO_CARD')
  #devoluciones['RFC'] = devoluciones['RFC'].fillna('NO_RFC')
meses_sin_intereses['TOTAL'] = meses_sin_intereses['TOTAL'].astype(float)
meses_sin_intereses['PARCIAL'] = meses_sin_intereses['PARCIAL'].astype(float)
meses_sin_intereses['RESTANTE'] = meses_sin_intereses['RESTANTE'].astype(float)
mensualidades['CANTIDAD'] = mensualidades['CANTIDAD'].astype(float)
pago_tarjeta['CANTIDAD'] = pago_tarjeta['CANTIDAD'].astype(float)

Total restante que te queda por pagar de todos tus meses sin intereses, no todo es a pagar este mes

In [ ]:
meses_sin_intereses['RESTANTE'].sum()

Total de compras que realizaste este mes

In [ ]:
gastos['CANTIDAD'].sum()

Total de pagos a tarjeta hechos por devoluciones

In [ ]:
if len(devoluciones) > 0:
  devoluciones['CANTIDAD'].sum()

Total de todas las mensualidades a pagar este mes

In [ ]:
mensualidades['CANTIDAD'].sum()

Total de pagos a tarjeta realizados, usualmente es lo que pagaste para saldar tu tarjeta en el periodo pasado

In [ ]:
pago_tarjeta['CANTIDAD'].sum()

Lista de gastos del período de mayor a menor agrupados por RFC y descripción

In [ ]:
gastos.groupby(['RFC', 'description'])['CANTIDAD'].sum().sort_values(ascending=False)

Pagos realizados a capital

In [ ]:
pago_tarjeta

# 6. Intentando categorizar cada gasto utilizando la descripción o concepto del gasto

Aquí se puede añadir a cada caso más descripciones si tus gastos no corresponden a los que ves

Por ejemplo si en el renglón 6 tienes un gasto que crees que debería ser etiquetado como RESTAURANTES y tiene como descripción 'SANBORNS', puedes agregarlo al final del resto en el renglón 6 como:

> 'SANBORNS' in row['description']



In [ ]:
for i, row in gastos.iterrows():
  if 'MERCADO' in row['description']:
    gastos.loc[i, 'LABEL'] = 'MERCADO LIBRE'
  elif 'AMAZON' in row['description']:
    gastos.loc[i, 'LABEL'] = 'AMAZON'
  elif 'CASSAVA' in row['description'] or 'TORTAS' in row['description'] or 'LITTLE CAESARS' in row['description'] or 'GARABATOS' in row['description'] or 'MI CABANA' in row['description'] or 'COMIDA RAPIDA' in row['description'] or 'TIERRA' in row['description'] or 'PLAZA ARTZ' in row['description'] or 'HABIA UNA RES' in row['description'] or 'UBER EATS' in row['description'] or 'STARBUCKS' in row['description']:
    gastos.loc[i, 'LABEL'] = 'RESTAURANTES'
  elif 'TELLO' in row['description'] or 'ATT' in row['description'] or 'CFE' in row['description'] or 'IZZI' in row['description'] or 'FINANZAS' in row['description'] or 'NETFLIX' in row['description']:
    gastos.loc[i, 'LABEL'] = 'SERVICIOS'
  elif 'APPLE' in row['description']:
    gastos.loc[i, 'LABEL'] = 'APPSTORE'
  elif 'DESPEGAR' in row['description'] or 'VOLARIS' in row['description']:
    gastos.loc[i, 'LABEL'] = 'VUELOS'
  elif 'EST ' in row['description'] or 'GASOL' in row['description']:
    gastos.loc[i, 'LABEL'] = 'GASOLINA'
  elif 'TUTAG' in row['description'] or 'LLANTAS' in row['description']:
    gastos.loc[i, 'LABEL'] = 'AUTO'
  elif 'MAYORISTAS' in row['description'] or 'GANDHI' in row['description'] or 'WAL MART' in row['description'] or 'MERPAGO' in row['description'] or 'PAYPAL' in row['description'] or 'aliexpress' in row['description']:
    gastos.loc[i, 'LABEL'] = 'COMPRAS'
  elif 'PETCO' in row['description']:
    gastos.loc[i, 'LABEL'] = 'GATOS'
  elif 'F AHORRO' in row['description'] or 'UROPED' in row['description'] or 'ATRYS' in row['description']:
    gastos.loc[i, 'LABEL'] = 'MEDIC'
  elif 'IKANO' in row['description']:
    gastos.loc[i, 'LABEL'] = 'IKEA'
  elif 'BERSHKA' in row['description']:
    gastos.loc[i, 'LABEL'] = 'ROPA'
  elif 'FERRETERIAS' in row['description']:
    gastos.loc[i, 'LABEL'] = 'CASA'
  elif 'SUPERCREMERIA' in row['description']:
    gastos.loc[i, 'LABEL'] = 'ABARROTES'
  elif 'GOOGLE' in row['description'] or 'HUGGING' in row['description'] or 'OPENAI' in row['description']:
    gastos.loc[i, 'LABEL'] = 'TECH'
  elif 'ABONO POR TRASP' in row['description']:
    gastos.loc[i, 'LABEL'] = 'TRASPASO'
  else:
    gastos.loc[i, 'LABEL'] = 'OTROS'


# 7. Graficando los rubros por gasto mensual

In [ ]:
x_ticks = []
for l in gastos.groupby(['LABEL'])['CANTIDAD'].sum().sort_values(ascending=False).reset_index()['LABEL'].to_list():
  x_ticks.append(gastos.loc[gastos['LABEL'] == l, 'LABEL'].to_list()[0])

to_plot = gastos.groupby(['LABEL'])['CANTIDAD'].sum().sort_values(ascending=False)
to_plot.index = x_ticks
to_plot.plot(kind='bar', rot=85)

Visualizando gastos por RFC

In [ ]:
x_ticks = []
for l in gastos.groupby(['RFC'])['CANTIDAD'].sum().sort_values(ascending=False).reset_index()['RFC'].to_list():
  x_ticks.append(gastos.loc[gastos['RFC'] == l, 'RFC'].to_list()[0])

to_plot = gastos.groupby(['RFC'])['CANTIDAD'].sum().sort_values(ascending=False)
to_plot.index = x_ticks
to_plot.plot(kind='bar', rot=85)

# 8. Gráfica de pastel de los mismos rubros

In [ ]:
to_plot.plot.pie(y='CANTIDAD', figsize=(10, 10))

# 9. Lista de gastos por rubro

In [ ]:
gastos.groupby(['LABEL'])['CANTIDAD'].sum().sort_values(ascending=False)

# 10. Si ves que hay demasiado en OTROS, revisa ese rubro y añade las reglas como se indica en el punto 6 para colocarlos en su categoría correcta

In [ ]:
gastos.loc[gastos['LABEL'] == 'OTROS']